<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_amarillo@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio ETL: Análisis del Sistema Energético en España

## Objetivo

Durante todos los laboratorios de esta semana realizarás un proceso completo de ETL para analizar la relación entre la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de un año. Además, complementarán este análisis con datos demográficos y económicos extraídos del Instituto Nacional de Estadística (INE). El **objetivo principal** del análisis es **examinar cómo la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de los años están influenciados por factores demográficos y económicos, como la población y el PIB provincial**. El análisis busca identificar patrones y correlaciones entre estas variables para comprender mejor las dinámicas energéticas regionales y su relación con el desarrollo socioeconómico en España.

Antes de realizar el análisis, vamos a definir las hipótesis con las que vamos a trabajar, las cuales definirán todo tu análisis y planteamiento de los laboratorios: 

- **Hipótesis 1: La demanda eléctrica está correlacionada con la población de la provincia.** Provincias con mayor población tienden a tener una mayor demanda eléctrica.
  
- **Hipótesis 2: El crecimiento económico (medido por el PIB) está correlacionado con el consumo eléctrico.** Las provincias con un PIB más alto o en crecimiento experimentan un mayor consumo de energía.

- **Hipótesis 3: La proporción de generación renovable está relacionada con factores económicos o geográficos.** Provincias con un mayor desarrollo económico o con condiciones geográficas favorables (como más horas de sol o viento) tienden a generar más energía renovable.


## Tareas Laboratorio Transformación

En este laboratorio, tu objetivo será limpiar y preparar los datos extraídos previamente de diferentes fuentes para su posterior análisis. Trabajarás con datos provenientes de la API de Red Eléctrica Española (REE) y del Instituto Nacional de Estadística (INE). Estos datos incluyen información sobre demanda y generación eléctrica a nivel provincial, así como datos demográficos y económicos. 


- Cargar los Datos Extraídos:

  - **Demanda Eléctrica:** Carga los datos de demanda eléctrica extraídos de la API de REE.

  - **Generación Eléctrica:** Carga los datos de generación eléctrica diferenciados por tipo de energía (eólica, solar, hidroeléctrica, etc.) a nivel provincial.

  - **Datos Demográficos:** Carga los datos demográficos por provincia extraídos de la web del INE.

  - **Datos Económicos:** Carga los datos del PIB por provincia obtenidos del INE.


-Limpieza de Datos:

- Datos de la API de REE:

  - **Demanda Eléctrica:**

    - **Conversión de Timestamps:** Asegúrate de que las fechas estén correctamente formateadas en `datetime`. Si es necesario, convierte los datos a un formato uniforme (por ejemplo, `YYYY-MM` para datos mensuales).

    - **Tratamiento de Valores Nulos:** Identifica y maneja los valores nulos en caso de que los haya. Puedes optar por eliminar filas con valores faltantes.

    - **Estandarización de Nombres de Provincias:** Verifica que los nombres de las provincias estén estandarizados y coincidan en todos los conjuntos de datos. Si hay inconsistencias, corrígelas.

  - **Generación Eléctrica:**

    - **Desagregación de Tecnologías:** Asegúrate de que los datos estén correctamente desglosados por tipo de energía. Revisa que los campos correspondientes a energía eólica, solar, hidroeléctrica, etc., estén bien identificados y sin errores.

    - **Normalización de Unidades:** Verifica que las unidades de energía estén estandarizadas (por ejemplo, MWh). Realiza las conversiones necesarias si se encuentran en otras unidades.

    - **Identificación de Outliers:** Revisa los valores extremos o atípicos en la generación de energía y decide si deben ser tratados o eliminados.

- Datos del INE:

  - **Datos Demográficos:**

    - **Consistencia en la Codificación de Provincias:** Asegúrate de que los nombres de las provincias en los datos demográficos coincidan con los nombres utilizados en los datos eléctricos.

    - **Revisión de Categorías:** Verifica que las categorías de edad, sexo, y nacionalidad estén correctamente etiquetadas y sean consistentes en todo el dataset.

    - **Manejo de Valores Faltantes:** Revisa la presencia de valores faltantes y decide cómo tratarlos (relleno, eliminación o sustitución).

  - **Datos Económicos:**

    - **Normalización del PIB:** Si los datos del PIB están en diferentes unidades o escalas, asegúrate de normalizarlos para que sean comparables entre provincias.

    - **Agrupación Temporal:** Si los datos económicos están disponibles en diferentes periodos temporales, agrúpalos y normalízalos para que coincidan con los datos eléctricos en términos de granularidad temporal (mensual o anual).

NOTA: Ten en cuenta que los datos los vamos a tener que insertar en una base de datos mañana, por lo que toda esta limpieza os recomendamos que la penséis para poder crear e insertar los datos mañana. 

In [16]:
import pandas as pd
import numpy as np
import json
import os

import warnings
warnings.filterwarnings('ignore')

In [17]:
lista_anios = [2019,2020,2021]

cod_comunidades = {'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20,
                    'Ceuta': 8744}

In [18]:
os.makedirs("datos/DataFrameFinales", exist_ok=True)

### Creación del dataframe de las comunidades autónomas

In [19]:
df_comunidades = pd.DataFrame.from_dict(cod_comunidades, orient='index')
df_comunidades.reset_index(inplace=True)
df_comunidades.columns=["nombre", "id_ccaa"]
df_comunidades = df_comunidades.reindex(columns=["id_ccaa", "nombre"])
df_comunidades.to_csv("datos/DataFrameFinales/df_comunidades.csv")

### Obtención y limpieza de tablas de demanda

In [20]:
df_demanda= pd.DataFrame()

for anio in lista_anios:
    df_anio = pd.DataFrame()
    for ccaa, id in cod_comunidades.items():
        try:
            with open(f"datos/evolucion/{anio}/{ccaa}.json") as f:
                data = json.load(f)

            included = data['included']
            values = included[0]['attributes']['values']

            df = pd.DataFrame(values)
            df["anio"] = anio
            df["ccaa"] = ccaa
            df_anio = pd.concat([df_anio, df])
            
        except:
            print(f"Ha fallado en el año {anio} y ccaa: {ccaa}")
            pass

    df_anio.to_csv(f"datos/evolucion/{anio}/df_final_{anio}.csv")

    df_demanda= pd.concat([df_demanda, df_anio])

df_demanda.to_csv("datos/evolucion/df_demanda_concatenado.csv")


Una vez tenemos el dataframe llevamos a cabo una limpieza más profunda:

In [21]:
df_demanda = pd.read_csv("datos/evolucion/df_demanda_concatenado.csv", index_col=0)
df_demanda.head(1)

,value,percentage,datetime,anio,ccaa
0,18530.186,1,2019-01-01T00:00:00.000+01:00,2019,Melilla


In [22]:
def obtener_mes(df, col_fecha):
    meses_dic = {
        "January": "Enero",
        "February": "Febrero",
        "March": "Marzo",
        "April": "Abril",
        "May": "Mayo",
        "June": "Junio",
        "July": "Julio",
        "August": "Agosto",
        "September": "Septiembre",
        "October": "Octubre",
        "November": "Noviembre",
        "December": "Diciembre"
    }

    df[col_fecha] = (pd.to_datetime(df[col_fecha].str.split("T", expand=True).get(0))).dt.month_name()
    df[col_fecha] = df[col_fecha].map(meses_dic)

    return df

In [23]:
# Modificamos el datetime y nos quedamos solo con el mes
df_demanda = obtener_mes(df_demanda, "datetime")
df_demanda.head(2)

,value,percentage,datetime,anio,ccaa
0,18530.186,1,Enero,2019,Melilla
1,15963.180,1,Febrero,2019,Melilla


In [24]:
def limpieza_basica(df):   
    # Cambiamos la ccaa por su id
    df = df.merge(df_comunidades, left_on="ccaa", right_on="nombre").drop(columns=["ccaa", "nombre"])

    # Renombramos las columnas
    df.rename(columns={"value":"valor", "datetime":"mes"}, inplace=True)

    return df

In [25]:
df_demanda = limpieza_basica(df_demanda)

# Eliminamos la columnas porcentaje
df.drop(columns="percentage", inplace=True)

# Reordenamos las columnas
df_demanda = df_demanda.reindex(columns=["id_ccaa", "anio", "mes", "valor"])

df_demanda.head(2)

,id_ccaa,anio,mes,valor
0,8745,2019,Enero,18530.186
1,8745,2019,Febrero,15963.180


In [26]:
# Comprobamos que no haya nulos
print(df_demanda.isna().sum())

# Comprobamos que no haya duplicados
print(f"Hay {df_demanda.duplicated().sum()} duplicados")

id_ccaa    0
anio       0
mes        0
valor      0
dtype: int64
Hay 0 duplicados


Una vez limpio lo guardamos en la carpeta de los dataframe finales:

In [27]:
df_demanda.to_csv("datos/DataFrameFinales/df_demanda.csv")

### Obtención y limpieza de tablas de generación electrica

In [28]:
df_final= pd.DataFrame()

for anio in lista_anios:
    df_anio = pd.DataFrame()
    for ccaa, id in cod_comunidades.items():
        try:
            with open(f"datos/generacion/{anio}/{ccaa}.json") as f:
                data = json.load(f)

            included = data['included']
            for elem in included:
                tipo_energia = elem["type"]
                valores = elem['attributes']['values']

                df = pd.DataFrame(valores)
                df["tipo"] = tipo_energia
                df["anio"] = anio
                df["ccaa"] = ccaa
                df_anio = pd.concat([df_anio, df])
                

        except:
            print(f"Ha fallado en el año {anio} y ccaa: {ccaa}")
            pass

    df_anio.to_csv(f"datos/generacion/{anio}/df_final_{anio}.csv")

    df_final= pd.concat([df_final, df_anio])

df_final.to_csv(f"datos/generacion/df_generacion_concatenado.csv")

Ha fallado en el año 2019 y ccaa: Ceuta
Ha fallado en el año 2020 y ccaa: Ceuta
Ha fallado en el año 2021 y ccaa: Ceuta


Una vez tenemos el dataframe llevamos a cabo una limpieza más profunda:

In [29]:
df_generacion = pd.read_csv("datos/generacion/df_generacion_concatenado.csv", index_col=0)
df_generacion.head(1)

,value,percentage,datetime,tipo,anio,ccaa
0,5.053,0.009113,2019-01-01T00:00:00.000+01:00,Solar fotovoltaica,2019,Melilla


In [30]:
# Modificamos el datetime y nos quedamos solo con el mes
df_generacion = obtener_mes(df_generacion, "datetime")

# Llevamos a cabo una limpieza básica
df_generacion = limpieza_basica(df_generacion)
df_generacion.head(2)


,valor,percentage,mes,tipo,anio,id_ccaa
0,5.053,0.009113,Enero,Solar fotovoltaica,2019,8745
1,5.526,0.011159,Febrero,Solar fotovoltaica,2019,8745


Debemos eliminar también las filas cuyo valor en tipo es "Generación renovable" porque hace referencia al total de energías renovables.

In [31]:
df_generacion = df_generacion[df_generacion["tipo"] != "Generación renovable"]

Necesitamos generar un dataframe para los tipos de energía y su id para poder sustitur la energia por su id en nuestro df_generacion.

In [32]:
df_energia = pd.DataFrame(df_generacion["tipo"].unique())

df_energia.index += 1
df_energia.reset_index(inplace=True)
df_energia=df_energia.rename(columns={"index":"id_energia", 0:"nombre"})
df_energia.to_csv("datos/DataFrameFinales/df_energia.csv")

Ahora ya podemos sustituir el tipo de energía por su id:

In [33]:
# Sustituimos las energías por su índice
df_generacion = df_generacion.merge(df_energia, left_on="tipo", right_on="nombre").drop(columns=["tipo", "nombre"])

# Reordenamos las columnas
df_generacion = df_generacion.reindex(columns=["id_ccaa", "anio", "mes", "id_energia", "percentage", "valor"])
df_generacion.rename(columns={"percentage":"porcentaje"}, inplace=True)
df_generacion.head(2)

,id_ccaa,anio,mes,id_energia,porcentaje,valor
0,8745,2019,Enero,1,0.009113,5.053
1,8745,2019,Febrero,1,0.011159,5.526


Definimos una función para comprobar que el campo de porcetaje para cada comunidad autónoma por año y mes cumpla el 100% entres sus diferentes energias renovables:

In [34]:
def check_porcentaje(df_generacion):
    lista_meses = ["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]
    lista_ccaa = list(df_generacion["id_ccaa"].unique())
    lista_anios = list(df_generacion["anio"].unique())

    for anio in lista_anios:
        for id_ccaa in lista_ccaa:
            for mes in lista_meses:
                df_ccaa_mes = df_generacion[(df_generacion["anio"] == anio) & (df_generacion["id_ccaa"] == id_ccaa) & (df_generacion["mes"] == mes)]
                df_ccaa_mes["total"]=df_ccaa_mes["valor"].sum()
                df_ccaa_mes
                for index, row in df_ccaa_mes.iterrows():
                    perc=round(row["valor"]/row["total"],6)
                    comprobacion = perc==round(row["porcentaje"],6)
                    if comprobacion == False:
                        id_energia = row["id_energia"]
                        print(f"El porcentaje esta mal para {id_ccaa}, {anio}, {mes}, {id_energia}")
                        df_generacion.loc[index, "porcentaje"] = perc


In [35]:
check_porcentaje(df_generacion)

El porcentaje esta mal para 8745, 2021, Diciembre, 1
El porcentaje esta mal para 8745, 2021, Diciembre, 2


In [36]:
# Comprobamos que no haya nulos
print(df_generacion.isna().sum())

# Comprobamos que no haya duplicados
print(f"Hay {df_generacion.duplicated().sum()} duplicados")

id_ccaa       0
anio          0
mes           0
id_energia    0
porcentaje    0
valor         0
dtype: int64
Hay 0 duplicados


Finalmente lo almacenamos en nuestra carpeta de dataframes limpios.

In [37]:
df_generacion.to_csv("datos/DataFrameFinales/df_generacion.csv")

# Datos del INE

## Tabla de población

In [148]:
df = pd.read_csv("datos/DatosDescargados/ine_poblacion.csv", sep=";", encoding='latin-1')

Procedemos a limpiar el dataframe:

In [149]:
def limpieza_provincias(df_filtrado):

    # Vamos a limpiar la columna provincia para quitar el id:
    df_filtrado["Provincias"]=df_filtrado["Provincias"].apply(lambda x: x.split()[1])

    reemplazos = {"Balears,": "Baleares", "Palmas,": "Las Palmas", "Coruña,": "A Coruña", "Ciudad":"Ciudad Real", "Rioja,":"La Rioja", "Santa": "Santa Cruz de Tenerife"}
    df_filtrado["Provincias"] = df_filtrado["Provincias"].replace(reemplazos)

    df_filtrado["Provincias"]=df_filtrado["Provincias"].apply(lambda x: x.split("/")[0])

    return df_filtrado


In [150]:
# Quitamos las filas que no aportan valor
df_poblacion = df[(df['Provincias'] != 'TOTAL ESPAÑA') & (df['Edad (3 grupos de edad)'] != 'TOTAL EDADES') & (df['Españoles/Extranjeros'] != 'TOTAL') & (df['Españoles/Extranjeros'] != '% Extranjeros') & (df['Sexo'] != 'Ambos sexos')]

df_filtrado = limpieza_provincias(df_poblacion)

In [151]:
df_filtrado.shape

(1872, 6)

Para el grupo de edades los vamos a categorizar en tres grupos y reemplzar los valores de dicha columna usando el diccionario dic_edades:

In [152]:
dic_edades={"0-15 años": "jovenes", "16-64 años": "adultos", "65 y más":"mayores"}
df_filtrado["Edad (3 grupos de edad)"]=df_filtrado["Edad (3 grupos de edad)"].map(dic_edades)

Para cada provincia vamos a sacar el codigo de su ccaa:

In [153]:
dic_provincias = {
    "Andalucía": ["Almería", "Cádiz", "Córdoba", "Granada", "Huelva", "Jaén", "Málaga", "Sevilla"],
    "Aragón": ["Huesca", "Teruel", "Zaragoza"],
    "Principado de Asturias": ["Oviedo", "Asturias"],
    "Illes Balears": ["Baleares"],
    "Canarias": ["Santa Cruz de Tenerife", "Las Palmas"],
    "Cantabria": ["Santander", "Cantabria"],
    "Castilla - La Mancha": ["Albacete", "Ciudad Real", "Cuenca", "Guadalajara", "Toledo"],
    "Castilla y León": ["Ávila", "Burgos", "León", "Palencia", "Salamanca", "Segovia", "Soria", "Valladolid", "Zamora"],
    "Cataluña": ["Barcelona", "Girona", "Lleida", "Tarragona"],
    "Comunitat Valenciana": ["Alicante", "Castellón", "Valencia"],
    "Extremadura": ["Badajoz", "Cáceres"],
    "Galicia": ["A Coruña", "Lugo", "Ourense", "Pontevedra"],
    "Comunidad de Madrid": ["Madrid"],
    "Región de Murcia": ["Murcia"],
    "Comunidad Foral de Navarra": ["Pamplona", "Navarra"],
    "País Vasco": ["Araba", "Bizkaia", "Gipuzkoa"],
    "La Rioja": ["Logroño", "La Rioja"],
    "Melilla":["Melilla"],
    "Ceuta":["Ceuta"]

}


In [154]:
lista_provincias = list(df_filtrado["Provincias"].unique())
lista_final = []
for provincia in lista_provincias:
    flag=0
    for comunidad, lista in dic_provincias.items():
        if provincia in lista:
            lista_final.append([provincia, comunidad])
            flag=1
            break
    if flag==1:
        continue
    print(provincia)

Creamos un dataframe para las provincias con su id y el de su ccaa.

In [220]:
df_provincias = pd.DataFrame(lista_final)
df_provincias.index += 1
df_provincias.reset_index(inplace=True)

In [221]:
df_provincias.rename(columns={0:"nombre", 1:"ccaa", "index":"id_provincia"}, inplace=True)

In [222]:
df_provincias_final = df_provincias.merge(how="left", right =df_comunidades, left_on="ccaa", right_on="nombre").drop(columns=["ccaa", "nombre_y"])
df_provincias_final.rename(columns={"nombre_x":"nombre"}, inplace=True)

In [223]:
df_provincias_final.to_csv("datos/DataFrameFinales/df_provincias.csv")

Ahora que ya tenemos el dataframe de provincias, sustitumos la provincia por su id y añadimos el id de ccaa.

In [159]:
df_provincias = pd.read_csv("datos/DataFrameFinales/df_provincias.csv", index_col=0)
df_provincias.head(1)

,id_provincia,nombre
0,1,Albacete


In [160]:
df_filtrado = df_filtrado.merge(how="left", right =df_provincias, left_on="Provincias", right_on="nombre").drop(columns=["Provincias", "nombre"])

Ahora crearemos un dataframe para los grupos de edades y sustituremos el valor de la tabla por el id.

In [161]:
df_edades = pd.DataFrame(df_filtrado["Edad (3 grupos de edad)"].unique())
df_edades.index += 1
df_edades.reset_index(inplace=True)
df_edades.rename(columns={"index":"id_edad", 0:"nombre"}, inplace=True)

df_edades.to_csv("datos/DataFrameFinales/df_edades.csv")

In [162]:
df_filtrado = df_filtrado.merge(how="left", right =df_edades, left_on="Edad (3 grupos de edad)", right_on="nombre").drop(columns=["Edad (3 grupos de edad)", "nombre"])

Renombramos y ordenamos las columnas:

In [163]:
df_filtrado["Total"]=df_filtrado["Total"].str.replace(".", "").astype(int)
df_filtrado = df_filtrado.reindex(columns=['id_provincia', 'id_edad', 'Españoles/Extranjeros', 'Sexo', 'Año', 'Total'])

In [164]:
# Comprobamos que no haya nulos
print(df_filtrado.isna().sum())

# Comprobamos que no haya duplicados
print(f"Hay {df_filtrado.duplicated().sum()} duplicados")

id_provincia             0
id_edad                  0
Españoles/Extranjeros    0
Sexo                     0
Año                      0
Total                    0
dtype: int64
Hay 0 duplicados


In [165]:
df_filtrado.to_csv("datos/DataFrameFinales/df_poblacion.csv")

## Tabla de PIB

In [181]:
df_pib = pd.read_csv("datos/DatosDescargados/ine_pib.csv", sep=";", encoding='latin-1')
df_pib

,Provincias,Ramas de actividad,periodo,Total
0,02 Albacete,PRODUCTO INTERIOR BRUTO A PRECIOS DE MERCADO,2021(P),8.663.814
1,02 Albacete,PRODUCTO INTERIOR BRUTO A PRECIOS DE MERCADO,2020,7.855.556
2,02 Albacete,PRODUCTO INTERIOR BRUTO A PRECIOS DE MERCADO,2019,8.475.070
3,02 Albacete,"A. Agricultura, ganadería, silvicultura y pesca",2021(P),884.324
4,02 Albacete,"A. Agricultura, ganadería, silvicultura y pesca",2020,768.976
...,...,...,...,...
1711,52 Melilla,Impuestos netos sobre los productos,2020,132.490
1712,52 Melilla,Impuestos netos sobre los productos,2019,151.213
1713,52 Melilla,PRODUCTO INTERIOR BRUTO A PRECIOS DE MERCADO,2021(P),1.616.543
1714,52 Melilla,PRODUCTO INTERIOR BRUTO A PRECIOS DE MERCADO,2020,1.514.009


In [182]:
# Quitamos las filas que no aportan valor
df_pib = df_pib[(df_pib['Ramas de actividad'] != 'PRODUCTO INTERIOR BRUTO A PRECIOS DE MERCADO') ]

# Quitamos el id de las provincias
df_filtrado = limpieza_provincias(df_pib)

Cambiamos el formato de algunas columnas:

In [183]:
df_filtrado["periodo"]=df_filtrado["periodo"].replace("2021(P)", "2021")
df_filtrado["Total"]=df_filtrado["Total"].str.replace(".", "").astype(int)

Añadimos el id de provincia:

In [184]:
df_provincias = pd.read_csv("datos/DataFrameFinales/df_provincias.csv", index_col=0)
df_filtrado = df_filtrado.merge(how="left", right =df_provincias, left_on="Provincias", right_on="nombre").drop(columns=["Provincias", "nombre"])

Reordenamos columnas:

In [185]:
df_filtrado = df_filtrado.reindex(columns=['id_provincia', 'Ramas de actividad', 'periodo', 'Total'])

Vamos a extraer otro dataframe ya que en las ramas hay dos valores: Valor añadido bruto total y Impuestos netos sobre los productos, que al sumarlos nos dan el pib. Y las ramas son una parte del total del pib, por lo que nos conviene tenerlo en dos dataframes separados.

In [197]:
filtro_ramas = (df_filtrado["Ramas de actividad"]=="Valor añadido bruto total") | (df_filtrado["Ramas de actividad"]=="Impuestos netos sobre los productos")
df_pib_total = df_filtrado[filtro_ramas]
df_pib_total.to_csv("datos/DataFrameFinales/df_pib_total.csv")

In [199]:
df_pib_ramas = df_filtrado[~filtro_ramas]

En el dataframe de pib_ramas vamos a crear un df de ramas y sustituir el valor en la tabla por su id

In [207]:
df_ramas = pd.DataFrame(df_pib_ramas["Ramas de actividad"].unique())
df_ramas.index += 1
df_ramas.reset_index(inplace=True)
df_ramas.rename(columns={"index":"id_rama", 0:"nombre"}, inplace=True)
df_ramas.to_csv("datos/DataFrameFinales/df_ramas.csv")

Vamos ahora a sutitir la rama por su id:

In [209]:
df_pib_ramas = df_pib_ramas.merge(how="left", right =df_ramas, left_on="Ramas de actividad", right_on="nombre").drop(columns=["Ramas de actividad", "nombre"])

In [211]:
df_pib_ramas=df_pib_ramas.reindex(columns=['id_provincia', 'id_rama', 'periodo', 'Total'])

In [212]:
df_pib_ramas

,id_provincia,id_rama,periodo,Total
0,1,1,2021,884324
1,1,1,2020,768976
2,1,1,2019,791464
3,1,2,2021,1397006
4,1,2,2020,1194438
...,...,...,...,...
1087,52,6,2020,254596
1088,52,6,2019,258800
1089,52,7,2021,783628
1090,52,7,2020,759737


In [213]:
# Comprobamos que no haya nulos
print(df_pib_ramas.isna().sum())

# Comprobamos que no haya duplicados
print(f"Hay {df_pib_ramas.duplicated().sum()} duplicados")

id_provincia    0
id_rama         0
periodo         0
Total           0
dtype: int64
Hay 0 duplicados


In [214]:
df_pib_ramas.to_csv("datos/DataFrameFinales/df_pib_ramas.csv")